In [5]:
mkdir -p /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github_Repo

In [15]:
cd /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/

/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack


In [9]:
!git clone https://github.com/deabfc/dp-promise

Cloning into 'dp-promise'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 78 (delta 21), reused 78 (delta 21), pack-reused 0 (from 0)
Receiving objects: 100% (78/78), 28.58 KiB | 9.53 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [ ]:
!python prepare_celeba.py --source /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/img_align_celeba --dest /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32 --width 32 --height 32

In [ ]:
!python prepare_celeba.py --source /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/img_align_celeba --dest /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64 --width 64 --height 64

In [16]:
ls

Data/               MIA_attack_DCTDiff.ipynb     move_data_output.log
DCTdiff_Github/     MIA_attack_DP-Promise.ipynb  move_data_output.log.stats
DP-Promise_Github/  move_data_error.log          move_data.sbatch


In [12]:
!python3 -m venv dp-promise-env
!source dp-promise-env/bin/activate

In [17]:
!pip install -r /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Obtaining opacus from git+https://github.com/deabfc/opacus_dp_promise.git@dp_promise#egg=opacus (from -r /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/requirements.txt (line 4))
  Cloning https://github.com/deabfc/opacus_dp_promise.git (to revision dp_promise) to ./src/opacus
  Running command git clone --filter=blob:none --quiet https://github.com/deabfc/opacus_dp_promise.git /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/src/opacus
  Running command git checkout -b dp_promise --track origin/dp_promise
  Switched to a new branch 'dp_promise'
  M     opacus/scripts/compute_dp_sgd_privacy.py
  M     scripts/install_via_pip.sh
  M     scripts/pytorch_install.sh
  M     website/scripts/build_website.sh
  M     website/sphinx/source/index.rst
  branch 'dp_promise' set up to track 'origin/dp_promise'.
  Resolved https://github.com/deabfc/opacus_dp_promise.git to commit 21f11051a1ac52ec888e135d7e58

In [18]:
import os
import shutil

# --- CONFIGURATION ---
source_dir = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64'   # Source folder
output_dir_1 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train'  # 80% Train
output_dir_2 = '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Test'   # 20% Test

# --- CREATE OUTPUT DIRECTORIES IF THEY DON'T EXIST ---
os.makedirs(output_dir_1, exist_ok=True)
os.makedirs(output_dir_2, exist_ok=True)

# --- GET SORTED FILE LIST ---
all_files = sorted([
    f for f in os.listdir(source_dir)
    if os.path.isfile(os.path.join(source_dir, f))
])

# --- CALCULATE SPLIT INDEX ---
split_index = int(0.8 * len(all_files))

# --- SPLIT FILES SEQUENTIALLY ---
files_A = all_files[:split_index]   # 80%
files_B = all_files[split_index:]   # 20%

# --- COPY FILES ---
for f in files_A:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_1, f))

for f in files_B:
    shutil.copy2(os.path.join(source_dir, f), os.path.join(output_dir_2, f))

print(f"✅ Done. {len(files_A)} files → {output_dir_1}, {len(files_B)} files → {output_dir_2}")


✅ Done. 162079 files → /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train, 40520 files → /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Test


In [1]:
cd /rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github

/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github


In [23]:
import os
print("Total images 32:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32")))
print("Total images 32_Train:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train")))
print("Total images 32_Test:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Test")))
print("Total images 64:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64")))
print("Total images 64_Train:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train")))
print("Total images 64_Test:", len(os.listdir("/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Test")))

Total images 32: 202599
Total images 32_Train: 162079
Total images 32_Test: 40520
Total images 64: 202599
Total images 64_Train: 162079
Total images 64_Test: 40520


In [24]:
!python pretrain.py --config configs/vanilla/celeba_32/config.yaml --workdir checkpoints/vanilla --job-id Pretrain_celeba32

07/25/2025 02:27:32:INFO:Configuration: {'data': {'name': 'celeba', 'img_ch': 3, 'img_size': 32, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train', 'num_classes': 10, 'class_condition': False, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr': 0.0002, 'batch_size': 128, 'epochs': 50, 'ema_rate': 0.9999, 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 50, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}}
07/25/2025 02:27:32:INFO:Using seed: 42
07/25/2025 02:27:32:INFO:Load dataset celeba
07/25/2025 02:27:33:INFO:Num parameters: 35751939
07/25/2025 02:31:18:INFO:Epoch1: {'loss': 0.050669757319995054}
07/25/2025 02:34:12:INFO:Epoch2: {'loss': 0.03238056712999246}
07/25/2025 02:37:05:INFO:Epoch3: {'loss': 0.031280492045038216}
07/25/2025 02:39:57:INFO:Epoch4: {'loss': 0.03087975247744720

In [ ]:
!python pretrain.py --config configs/vanilla/celeba_64/config.yaml --workdir checkpoints/vanilla --job-id Pretrain_celeba64

07/25/2025 04:57:57:INFO:Configuration: {'data': {'name': 'celeba64', 'img_ch': 3, 'img_size': 64, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train', 'num_classes': 10, 'class_condition': False, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr': 0.0002, 'batch_size': 128, 'epochs': 50, 'ema_rate': 0.9999, 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 50, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}}
07/25/2025 04:57:57:INFO:Using seed: 42
07/25/2025 04:57:57:INFO:Load dataset celeba64
07/25/2025 04:57:59:INFO:Num parameters: 35751939
07/25/2025 05:15:51:INFO:Epoch1: {'loss': 0.03923027694442889}
07/25/2025 05:27:26:INFO:Epoch2: {'loss': 0.021981067379205924}
07/25/2025 05:39:01:INFO:Epoch3: {'loss': 0.021192528119997285}
07/25/2025 05:50:36:INFO:Epoch4: {'loss': 0.0204517202681

In [2]:
!python train.py --config configs/dp_promise/celeba_32/eps1.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba32_Eps1.0

07/25/2025 07:04:43:INFO:Configuration: {'data': {'name': 'celeba', 'img_ch': 3, 'img_size': 32, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/32_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 32, 'batch_size2': 4096, 'epochs1': 2, 'epochs2': 30, 'max_physical_batch_size': 16, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba32_celeba_0725-02:27:32/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 1.0, 'S': 900, 'sigma': 3.66}}
07/25/2025 07:04:44:IN

In [3]:
!python train.py --config configs/dp_promise/celeba_64/eps1.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba64_Eps1.0

07/26/2025 15:59:14:INFO:Configuration: {'data': {'name': 'celeba64', 'img_ch': 3, 'img_size': 64, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 16, 'batch_size2': 4096, 'epochs1': 1, 'epochs2': 15, 'max_physical_batch_size': 10, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba64_celeba64_0725-04:57:57/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 1.0, 'S': 950, 'sigma': 2.83}}
07/26/2025 15:59:1

In [4]:
!python train.py --config configs/dp_promise/celeba_64/eps10.0/config.yaml --workdir checkpoints/dp-promise --job-id Train_celeba64_Eps10.0

07/28/2025 09:55:24:INFO:Configuration: {'data': {'name': 'celeba64', 'img_ch': 3, 'img_size': 64, 'path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/Data/processed/64_Train', 'num_classes': 10, 'class_condition': True, 'num_workers': 2}, 'diffusion': {'timesteps': 1000, 'beta_start': 0.0001, 'beta_end': 0.02}, 'train': {'lr1': 0.0003, 'lr2': 0.0003, 'batch_size1': 16, 'batch_size2': 4096, 'epochs1': 1, 'epochs2': 15, 'max_physical_batch_size': 10, 'num_noise_sample': 4, 'ema_rate': 0.9999, 'ckpt_path': '/rds/projects/c/chenhp-dm-mia/Diff-MIA-Attack/DP-Promise_Github/checkpoints/vanilla/vanilla_Pretrain_celeba64_celeba64_0725-04:57:57/checkpoints/ema_50.ckpt', 'seed': 42}, 'sample': {'type': 'ddim', 'samplesteps': 200, 'rho': 1.0, 'batch_size': 256, 'guide_scale': 0.0, 'seed': 42}, 'model': {'ch': 128, 'ch_mult': [1, 2, 2, 2], 'num_res_blocks': 2, 'dropout': 0.0, 'attn': [2]}, 'dp': {'max_grad_norm': 0.01, 'delta': 1e-06, 'epsilon': 10.0, 'S': 950, 'sigma': 0.632}}
07/28/2025 09:55

In [2]:
ls

checkpoints/     MIA-Attack.ipynb     pretrain.py            src/
configs/         _models/             README.md              train.py
dp-promise-env/  prepare_celeba.py    requirements_eval.txt
evaluation/      prepare_imagenet.py  requirements.txt
